
# تکلیف NLP — گزارش

این گزارش، روایت کامل پروژه‌ی تکلیف پردازش زبان طبیعی فارسی را در حالی ارائه می‌کند که عمق فنی لازم برای بازتولید نتایج و گسترش کد حفظ شده است. متن حاضر توضیحی خوانا و انسانی را با جزئیات دقیق ماژول‌ها، پیکربندی‌ها، دستورها و نکته‌های مهمی که در راه‌اندازی و رفع اشکال به آن‌ها برخورد شد ترکیب می‌کند. گستره‌ی پروژه از تمرین‌های عبارات منظم برای تاریخ‌ها، اختصارنویسی‌ها، صفات HTML و JSON آغاز می‌شود، با مطالعه‌ی تطبیقی سه روش توکن‌سازی زیرواحدی ادامه پیدا می‌کند، یک لوله‌ی پیش‌پردازش عملی برای داده‌های موازی ترجمه را پی می‌گیرد، نمونه‌هایی از برچسب‌گذاری اجزای سخن و شناسایی موجودیت‌های نامدار را نشان می‌دهد و در نهایت با آموزش و ارزیابی یک مبنای ساده‌ی دنباله‌به‌دنباله مبتنی بر LSTM جمع‌بندی می‌شود. کد به‌عمد ماژولار و «پیکربندی‌محور» نوشته شده و گزینه‌ی «داکر-اول» برای یکسان‌سازی اجراها و کش‌کردن دانلودهای حجیم در دسترس است.


## مخزن و فلسفه‌ی طراحی

چینش مخزن به‌گونه‌ای است که مسئولیت‌ها را روشن و بازتولیدپذیری را تشویق کند. فایل‌های پیکربندی در مسیر `configs/` قرار گرفته‌اند تا همه‌ی انتخاب‌های متغیر—از الگوهای regex و تنظیمات توکن‌سازی گرفته تا آستانه‌های پیش‌پردازش و ابرپارامترهای مدل—بدون تغییر کد قابل تنظیم باشند. ماژول‌های منبع تحت `src/` و بر حسب نگرانی‌ها گروه‌بندی شده‌اند: «regex»، «tokenization»، «linguistics»، «preprocess»، «models»، «train»، «utils» و ورودی‌های کوچک «cli» که اجزاء را به هم متصل می‌کنند. داده‌ها به «خام»، «میانی» و «پردازش‌شده» تفکیک شده‌اند تا تأکید شود هر دگرگونی صریح و قابل ردیابی است، و گزارش‌ها و شکل‌ها در `reports/` نگه‌داری می‌شوند. فایل‌های داکر، compose و Makefile در ریشه قرار دارند تا جریان توسعه کوتاه و روشن بماند. این ساختار همان چیزی است که در کد پیاده‌سازی شده است.

اجرای کد از ریشه‌ی مخزن و با سبک فراخوانی ماژول‌ها—برای نمونه `python -m src.cli.run_seq2seq`—در نظر گرفته شده تا مسیرهای نسبیِ موجود در پیکربندی‌ها پایدار و قابل پیش‌بینی باشند. در صورت استفاده از داکر، تصویرها نسخه‌ی پایتون و کتابخانه‌ها را میخ‌کوب می‌کنند و کش‌های مدل برای Hugging Face، NLTK و Stanza روی ولوم‌های نام‌دار سوار می‌شوند تا میان ساخت‌های مجدد پایدار بمانند. همین رفتار را می‌توان به‌صورت محلی نیز با نصب چند وابستگی کوتاه از مسیر pip یا conda و یک‌بار دانلود مدل‌های فارسی Stanza به دست آورد.


## محیط و بازتولیدپذیری

هدف پروژه پایتون ۳.۱۰ یا جدیدتر است و روی ۳.۱۱ و ۳.۱۲ نیز آزموده شده است. بسته‌های ضروری شامل PyTorch برای مدل‌سازی، `transformers` برای توکنایزر WordPiece و لوله‌ی اختیاری NER، کتابخانه‌ی سطح پایین `tokenizers` برای آموزش BPE، `sentencepiece` برای مدل‌های unigram/BPE، یکی از `stanza` یا `hazm` برای POS و NER و نهایتاً `pyyaml` برای خواندن پیکربندی‌هاست. در نصب محلی می‌توان محیط را با چند فرمان کوتاه آماده کرد و برای Stanza یک‌بار مدل‌های فارسی را دریافت نمود. در بستر داکر همین اثر با یک build و کش‌های پایدار حاصل می‌شود. فایل compose دایرکتوریِ کاری را روی ریشه‌ی مخزن تنظیم می‌کند تا مسیرهای نسبیِ داخل `configs/` همان‌طور که نوشته شده‌اند قابل استفاده باشند. تصویرها کاربر غیر-root ایجاد می‌کنند تا مشکل مجوزها روی bind mount رخ ندهد و با یک init کوچک، سیگنال‌ها درست منتقل شود. هنگام اجرای نوت‌بوک‌ها، پروفایل compose سرویس Jupyter را در کانتینر بالا می‌آورد و بی‌نیاز از token یک پورت میزبان را باز می‌کند تا آزمایش‌ها روان‌تر شود.


## قراردادهای داده و پیش‌پردازش

جزء ترجمه‌ی ماشینی یک فایل TSV با کُدگذاری UTF-8 را انتظار دارد که در پیکربندیِ پیش‌پردازش نام‌گذاری شده و دقیقاً دارای سرستون‌های `src` و `tgt` است. کد پیش‌پردازش این فایل را از `data/raw` می‌خواند، نرمال‌سازی ویژه‌ی فارسی را اعمال می‌کند، بر اساس طولِ توکنی پالایش انجام می‌دهد، زوج‌های تکراری را حذف می‌کند، در صورت تنظیم زوج‌هایی را که مبدأ و مقصدشان یکسان است کنار می‌گذارد و سپس با یک seed ثابت، به‌صورت قطعی درهم‌ریزی انجام می‌دهد و در پایان `train.tsv`، `dev.tsv` و `test.tsv` را در `data/processed` می‌نویسد. نرمال‌سازی شامل نگاشت شکل عربی «ي/ك» به شکل فارسی «ی/ک»، کاهش نیم‌فاصله‌های پیاپی به یک نمونه‌ی منفرد با حفظ مرزهای مشروع، و فروکاستن فاصله‌های اضافی است. این انتخاب‌ها برای بهبود کیفیت مدل بدون زدودن تمایزهای املایی مهم اتخاذ شده‌اند. اگر فایل ورودی موجود نباشد، ابزار با پیامی صریح حاوی مسیر مورد انتظار و یادآوریِ سرستون‌ها خطا می‌دهد تا بازخورد راه‌اندازی سریع و بی‌ابهام باشد.


## عبارات منظم: دامنه و محدودیت‌ها

تمرین‌های regex به گونه‌ای طراحی شده‌اند که الگوهای مفید را پوشش دهند بدون آن‌که از regex برای مسائلی استفاده شود که واقعاً به پارسر نیاز دارند. استخراج تاریخ‌ها از سه دسته پشتیبانی می‌کند: تاریخ‌های کاملاً عددی با اسلش یا خط تیره و با ارقام فارسی یا لاتین مانند `۱۴۰۳/۰۶/۲۵` یا `1403-06-05`؛ قالب روز/نام‌ماه/سال که در آن نام ماه یکی از ماه‌های فارسی است؛ و قالب گفتاری که در آن یک عدد ترتیبی مانند «بیست و پنجم» پیش از نام ماه می‌آید و سپس عبارت «ماه سال» و یک سال چهاررقمی ذکر می‌شود. قالب آخر بر فهرستی محدود از واژه‌های ترتیبی تکیه دارد و در صورت نیاز به‌سادگی قابل گسترش است. شناسایی اختصارات روی نشانه‌هایی تمرکز دارد که در پرانتز می‌آیند مانند «(ص)»، «(ره)»، «(ق.م)» و «(س)» و مجموعه‌ی دقیق آن از فایل پیکربندی خوانده می‌شود تا بدون تغییر کد بتوان آن را افزود. استخراج شناسه‌ها و کلاس‌های HTML هر دو حالت تک‌نقل‌قول و دوجهت نقل‌قول را پوشش می‌دهد و کلاس‌های چندگانه را به توکن‌های منفرد می‌شکند. تشخیص JSON عامدانه محتاطانه است: ابتدا بررسی می‌شود که ورودی ظاهراً مرزهای JSON را دارد و سپس با `json.loads` واقعاً اعتبارسنجی می‌شود. پیام کلی این است که regex برای پالایش و استخراج عالی است و اعتبارسنجی باید به کتابخانه‌های متخصص همان دامنه سپرده شود.


## مطالعه‌ی توکنایزر و کاربری عملی

سه رویکرد زیرواحدی مقایسه شده‌اند زیرا در عمل با آن‌ها مواجه خواهید شد و روی متن فارسی رفتار متفاوتی دارند. WordPiece از طریق یک توکنایزر چندزبانه‌ی از پیش‌آموزش‌دیده با API مربوط به `AutoTokenizer` آزمایش می‌شود؛ این کار بی‌نیاز از آموزش است و بخش‌بندی‌ای را نشان می‌دهد که یک واژگان پرکاربرد تولید می‌کند. BPE به‌صورت محلی با کتابخانه‌ی `tokenizers` از Hugging Face و با «پیش‌توکن‌ساز فاصله‌ی سفید» برای سادگی آموزش داده می‌شود؛ کمک‌ابزار مربوطه پیش از ذخیره‌ی مدل JSON از وجود فایل‌های پیکره اطمینان حاصل می‌کند و دایرکتوری مقصد را می‌سازد. SentencePiece ارزشمند است چون نرمال‌سازی را یکپارچه می‌کند و نسبت به خط نوشته بی‌طرف است؛ روال آموزش در این پروژه «سخت‌گیریِ اندازه‌ی واژگان» را برمی‌دارد و اگر پیکره برای اندازه‌ی درخواستی کوچک باشد با مقادیر کوچک‌تر دوباره تلاش می‌کند تا از خطا جلوگیری و شما را به پیکربندی شدنی راهنمایی کند. اجرای فرمان مقایسه، رشته‌ی توکن‌ها را برای چند جمله چاپ می‌کند و آگاه‌کننده‌ترین مشاهدات مربوط به نحوه‌ی بخش‌بندی نیم‌فاصله، چگونگی برخورد با اعداد و واژه‌های مختلط فارسی–لاتین، بسامد توکن ناشناخته و نسبت رشد طول دنباله نسبت به خط‌پایه‌ی فاصله‌محور است. برای مدل ترجمه بهتر است یک روش را انتخاب کنید و در هر دو جهت ثابت نگه دارید؛ به‌کارگیری SentencePiece از ابتدا تا انتهای مسیر، گام عملی مناسبی است چون واژگانی فشرده و متناسب با داده‌ی خودتان می‌دهد.


## برچسب‌گذاری اجزای سخن و شناسایی موجودیت‌های نامدار

POS و NER با موتورهای قابل‌تعویض ارائه شده‌اند تا بتوانید از آنچه محیط‌تان پشتیبانی می‌کند بهره ببرید. اگر در پیکربندی Stanza را برگزینید و یک‌بار مدل‌های فارسی را دانلود کنید، این پایپ‌لاین توکن‌ها را همراه با برچسب‌های UPOS یا XPOS و در صورت فعال بودن، گستره‌های موجودیت با نوع آن‌ها برمی‌گرداند. اگر Hazm انتخاب شود، کد از توکنیزه‌کردن و برچسب‌زن POSِ Hazm استفاده می‌کند به شرطی که مسیر مدل را روی دیسک معرفی کنید. یک لوله‌ی NER مبتنی بر transformers نیز در دسترس است وقتی بخواهید از مدل Hugging Face با تجمیع ساده‌ی بازه‌ها استفاده کنید. فرمان نمایش، زوج‌های توکن–برچسب را برای یک جمله چاپ می‌کند؛ کد تعمداً در هم‌ترازی بین توکن‌ها و بازه‌های موجودیت ساده مانده تا تمرکز بر صحت سیم‌کشی و اعتبارسنجی محیط باشد. در سامانه‌ی تولیدی، هم‌ترازی باید بر مبنای اندیس‌های کاراکتری انجام شود نه مقایسه‌ی سطحی رشته‌ها.


## مبنای دنباله‌به‌دنباله

مدل مبنا یک کُدگذار–کُدگشا با embedding و LSTM در هر دو سو است. کُدگذار دنباله‌ی مبدأ را می‌گیرد و حالت‌های نهان و سلولیِ نهایی را برمی‌گرداند؛ کُدگشا از نشانه‌ی آغاز دنباله شروع می‌کند و گام‌به‌گام تولید می‌کند. «واداشتن معلم» بر اساس احتمالی که از پیکربندی می‌آید، حقیقتِ مبنا را با پیش‌بینی‌های مدل مخلوط می‌کند. آموزش با «آنتروپی متقاطع» و نادیده‌گرفتن اندیس پَد انجام می‌شود، گرادیان‌ها برای پایداری با هنجار ثابت بریده می‌شوند و به‌صورت پیش‌فرض بهینه‌ساز Adam به کار می‌رود. بهترین وزنه بر حسب خطای توسعه انتخاب و در شاخه‌ی مصنوعات ذخیره می‌شود. در ارزیابی، کد با «حریصانه» دیکود کردن تولید می‌کند و برای کنترل سلامت کار، یک BLEU ساده در سطح پیکره محاسبه می‌کند. پیاده‌سازی BLEU در این‌جا جریمه‌ی اختصار را اعمال نمی‌کند و هدفش کشف پسرفت‌هاست نه قیاس با ادبیات. کلاس داده‌مجموعه دنباله‌ها را به طول‌های ثابتی که در پیکربندی آمده پد می‌کند و سازنده‌ی واژگان اندیس‌هایی به نشانه‌های رزروشده‌ی پد/آغاز/پایان/ناشناخته اختصاص می‌دهد. انتخاب دستگاه به‌صورت خودکار انجام می‌شود: هرگاه CUDA در دسترس باشد ترجیح داده می‌شود، روی macOS از MPS استفاده می‌شود و در غیر این صورت اجرای CPU ادامه می‌یابد؛ فقدان `torch` با پیامی راهنما و دستور نصب مناسبِ همان سکو گزارش می‌شود.


## نتایج و تفسیر

با پیکره‌های کوچکِ آزمایشی، مدل سریع آموزش می‌بیند و خطا کاهش می‌یابد اما انتظار BLEU بالایی روی مجموعه‌ی توسعه واقع‌بینانه نیست. بهترین شیوه‌ی قضاوت در این مرحله بازبینی کیفی است؛ چند جمله‌ی مبدأ را کنار حدس‌های مدل و پاسخ‌های مرجع بگذارید و به دنبال الگوهای منظم خطا بگردید. تکرار توکن‌ها، پایان زودهنگام دنباله و آشفتگی ترتیب واژه‌ها در مبناهای LSTM ساده رایج‌اند و معمولاً با افزودن «توجه» یا مهاجرت به «ترنسفورمر» به‌وضوح بهبود می‌یابند. با بزرگ‌شدن پیکره‌ی موازی، ثبت دقیق فایل‌های پیکربندی، هش commit، خطاهای آموزش و توسعه در هر دوره و هر تغییرِ واردشده در پیش‌پردازش یا توکن‌سازی اهمیت می‌یابد تا نسبت دادن بهبودها به مداخله‌های مشخص ممکن باشد.


## عیب‌یابی و سخت‌افزاری‌سازی

چند مسئله در راه‌اندازی آشکار شد و در کد حل شده است. YAMLهایی که regex در خود دارند اگر نقل‌قول‌ها با دقت فرار داده نشوند ممکن است هنگام پارس خطا دهند؛ به‌کارگیری رشته‌های تک‌نقل‌قول در پیکربندی و دوبرابرکردن نقل‌قول‌های درونی ساده‌ترین راهکار بود. کمک‌ابزار آموزش BPE اکنون پیش از ذخیره‌ی مدل JSON از وجود دایرکتوری مقصد مطمئن می‌شود و در صورت مسیر اشتباه پیکره، پیام روشنی می‌دهد. آموزش SentencePiece دیگر وقتی اندازه‌ی واژگان درخواستی از ظرفیت پیکره‌ی کوچک فراتر می‌رود شکست نمی‌خورد؛ این روال «سخت‌گیری اندازه‌ی واژگان» را برمی‌دارد و با مقادیر کوچک‌تر دوباره تلاش می‌کند تا مدل شدنی حاصل شود، یا صریحاً اعلام می‌کند که پیکره حتی برای واژگان‌های کوچک کافی نیست. کد آموزش فقدان `torch` را با دستور نصب متناسب با سکو گزارش می‌کند. گام پیش‌پردازش نبود فایل TSV ورودی را با مسیر کامل و یادآوری سرستون‌ها اخطار می‌دهد. توابع POS و NER نیز نبود Stanza یا Hazm یا دانلودنشدن مدل‌های فارسی Stanza را با پیام‌های مستقیم گوشزد می‌کنند. این تدابیر کوچک، اجرای پروژه روی ماشین تازه و هم‌رسانی با همکار را بسیار آسان‌تر می‌کنند.


## دستورات قابل بازتولید

هرچند این متن از فهرست‌بندی پرهیز می‌کند، حفظ دستورهای دقیق برای بازتولید هر بخش مفید است. قطعه‌کدهای زیر با فرض مسیرهای پیش‌فرض در پیکربندی‌ها قابل کپی و اجرا از ریشه‌ی مخزن هستند و البته می‌توانید مسیرها و نام مدل‌ها را مطابق محیط خود تنظیم کنید.

```bash
python -m src.cli.run_regex --mode dates --text "۱۴۰۳/۰۶/۲۵"
python -m src.cli.run_regex --mode abbr  --text "(ق.م) و (ره)"
python -m src.cli.run_regex --mode html  --text '<div id="div1" class="a b"></div>'
python -m src.cli.run_regex --mode json  --text '{"a": 1}'
python -m src.cli.run_tokenizers --compare
python -m src.cli.run_tokenizers --compare --samples-file data/raw/sample_texts/fa_examples.txt
python -m src.cli.run_seq2seq --prep_cfg configs/preprocess.yml --model_cfg configs/model_seq2seq.yml
python -m src.cli.run_pos_ner --text "دیروز در تهران باران شدیدی بارید و محمد رضایی به دانشگاه تهران رفت."
```


## منطق پیکربندی

فایل‌های پیکربندی انتخاب‌های پروژه را به‌صورت شفاف کُدگذاری می‌کنند. پیکربندی regex شامل فهرست نام ماه‌های فارسی، مجموعه‌ی محدود واژه‌های ترتیبی برای قالب گفتاری و نحوه‌ی نقل‌قولی است که از دستبرد YAML در فراردهی‌ها جلوگیری می‌کند. پیکربندی توکن‌سازی نام مدل از پیش‌آموزش‌دیده‌ی WordPiece را مشخص می‌کند، مسیرهای پیکره و خروجی BPE را تعیین می‌کند و پارامترهای SentencePiece مانند نوع مدل، اندازه‌ی واژگان، پوشش کاراکتری و سخت‌گیری اندازه‌ی واژگان را می‌گذارد؛ کنار گذاشتن سخت‌گیری به‌ویژه در پیکره‌های کوچک مفید است. پیکربندی پیش‌پردازش مسیر TSV ورودی، خروجی‌های نرمال‌شده، حداقل و حداکثر طول برای پالایش، حذف تکراری‌ها و زوج‌های یکسان و نسبت‌های تقسیم و seed درهم‌ریزی را تعیین می‌کند. پیکربندی POS/NER موتور را بین Hazm، Stanza و لوله‌ی مبتنی بر transformers جابه‌جا می‌کند و پارامترهای لازم مانند زبان و پردازنده‌های Stanza یا مسیر مدل Hazm را دربر می‌گیرد. پیکربندی دنباله‌به‌دنباله ابعاد embedding و نهانِ کُدگذار و کُدگشا، نرخ واداشتن معلم، بهینه‌ساز و نرخ یادگیری، برش گرادیان، اندازه‌ی بچ و شمار دوره‌ها، طول‌های بیشینه‌ی دنباله، نشانه‌های ویژه‌ی پد و مرزهای دنباله و رفتار ذخیره‌ی بهترین وزن‌ها بر حسب خطای توسعه را تثبیت می‌کند.


## گام‌های بعدی

مسیر طبیعی بهبود مبنا، گذار به توکن‌های زیرواحدی در سراسر مسیر و افزودن «توجه» است. ارتقای سرراست این است که یک مدل SentencePiece روی متنِ ترکیبیِ مبدأ و مقصد با اندازه‌ی واژگان متناسب با پیکره آموزش دهید و مسیر را بازبسازید تا هر دو سوی مدل ترجمه همان طرح زیرواحدی را دریافت کنند. افزودن مکانیزم توجه—چه Bahdanau و چه Luong—افت کارایی روی دنباله‌های بلند را که در LSTM ساده رایج است جبران می‌کند و معمولاً بهبود فوری می‌آورد. در ادامه، جایگزینی معماری با یک ترنسفورمر کُدگذار–کُدگشا انتخاب امروزِ ترجمه‌ی عصبی است و با ادبیات روز هم‌خوان‌تر خواهد بود. در سمت ارزیابی، به‌کارگیری sacreBLEU و chrF++ نمره‌هایی قابل قیاس در اجراها و محیط‌های مختلف می‌دهد و افزودن beam search به دیکودینگ برآورد بهتری از توان بالقوه‌ی مدل فراهم می‌کند. با رشد پیکره، ارزش دارد آستانه‌های نرمال‌سازی و طول را بازنگری کنید و پوشش regex را با آزمون‌های واحد برای گونه‌های بیشتر تاریخ و اختصارات گسترش دهید؛ ساختار مخزن این تغییرها را مجزا و برگشت‌پذیر نگه می‌دارد.


## جمع‌بندی

این پروژه در جاه‌طلبی مدل‌سازی متواضع و در شفافیت مهندسی سخاوتمند است. مبنایی کارآمد برای پیش‌پردازش و ترجمه‌ی فارسی فراهم می‌کند، جایگاهی روشن برای اتصال توکن‌سازی و معماری‌های قوی‌تر می‌گذارد و مجموعه‌ای از ابزارهای کوچک و سرهم‌شدنی عرضه می‌کند که هر مرحله را قابل راستی‌آزمایی می‌سازند. ترکیب توضیح روایتی و پیکربندیِ دقیق که در این متن آمده، هم خوانا و هم عمل‌پذیر است تا بتوانید این سامانه را در گذر زمان نگه‌داری و آن را همگام با رشد داده‌ها و اهداف‌تان به مترجمی توانمندتر ارتقا دهید.
